# Getting Started

In this notebook, we will go through the basics of creating and `AIML` spec and using Guardrails to enforce it.

## Objective

Our goal is understand what a bank run is, and generate URL links to relevant news articles. We will first generate an `AIML` spec for this and then use Guardrails to enforce it.

## Installation

To get started, install the `guardrails` package with `pip`.


In [1]:
!pip install guardrails-ai

## Creating an `AIML` spec

At the heart of `Guardrails` is the `AIML` spec.

`AIML` a flavor of XML (standing for `AI Markup Language`) that describes the expected structure and type of the output of the LLM, the quality criteria for the output to be valid and corrective actions to be taken if the output is invalid.

- For this task, we create an `AIML` spec that requests the LLM to generate an object with 2 fields: `explanation` and `follow_up_url`.
- For the `explanation` field to be valid, the max length of the generated string should be **between 200 to 280 characters in length**. In case the generated string doesn't meet this criteria, for now we just want to log the validation error and continue (i.e. the **noop** action).
- For the `follow_up_url` field to be valid, the **URL should be reachable**. In case this quality criteria is not met, the generated output should be filtered out (i.e. the **filter** action).

Ordinarily, the `AIML` spec would be created in a file directly. However, for the purposes of this demo, we write the spec in a string and then create a file from it.

We specify our quality criteria (generated length, URL reachability) in the `format` fields of the `AIML` spec below. For now, we want to do nothing if the quality criteria for `explanation` is not met, and filter the `follow_up_url` if it is not valid.

In [2]:
aiml_spec = """
<aiml version="0.1">

<output>
    <object name="bank_run" format="length: 2">
        <string
            name="explanation"
            description="A paragraph about what a bank run is."
            format="length: 200 240"
            on-fail-length="noop"
        />
        <url
            name="follow_up_url"
            description="A web URL where I can read more about bank runs."
            required="true"
            format="valid-url"
            on-fail-valid-url="filter"
        />
    </object>
</output>

<prompt>
Explain what a bank run is in a tweet.

@xml_prefix_prompt

{output_schema}

@json_suffix_prompt_v2_wo_none
</prompt>
</aiml>
"""

## Using Guardrails to enforce the `AIML` spec

We write the `AIML` spec to a file and then use it to create a `Guard` object. The `Guard` object is used to wrap the LLM API call and enforce the `AIML` spec on the output of the LLM call.

In [3]:
import tempfile
from rich import print

import guardrails as gd

with tempfile.NamedTemporaryFile(mode="w", suffix=".aiml") as f:
    f.write(aiml_spec)
    f.flush()
    guard = gd.Guard.from_aiml(f.name)

We can see that the `Guard` object compiles the `AIML` output specification and adds it to the provided prompt.

In [4]:
print(guard.base_prompt)

Explain what a bank run is in a tweet.


Given below is XML that describes the information to extract from this document and the tags to extract it into.


<output>
    <object name="bank_run" format="length: 2">
        <string name="explanation" description="A paragraph about what a bank run is." format="length: 200 240"/>
        <url name="follow_up_url" description="A web URL where I can read more about bank runs." required="true" 
format="valid-url"/>
    </object>
</output>




ONLY return a valid JSON object (no other text is necessary). The JSON MUST conform to the XML format, including 
any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise.

JSON Output:

Next, we call the `Guard` object with the LLM API call as the first argument and add any additional arguments to the LLM API call as the remaining arguments.

In [5]:
import openai

# Wrap the OpenAI API call with the `guard` object
raw_llm_output, validated_output = guard(openai.Completion.create, engine="text-davinci-003", max_tokens=1024, temperature=0.3)

# Print the validated output from the LLM
print(validated_output)

{
    'bank_run': {
        'explanation': "A bank run is when a large number of customers withdraw their deposits from a bank due to 
concerns about the bank's solvency.",
        'follow_up_url': 'https://www.investopedia.com/terms/b/bankrun.asp'
    }
}

In [6]:
print(f'Len of explanation: {len(validated_output["bank_run"]["explanation"])}')

Len of explanation: 125

As we can see, the `explanation` field didn't meet the quality criteria (length between 200 and 280 characters). However, because of the the `noop` action specified in the `AIML` spec, the `Guard` object returned the output of the LLM API call as is.

Next, we change the `AIML` spec to reask the LLM for a correct `explanation` if its length is incorrect. We do this by creating a new `AIML` spec and creating a new `Guard` object.

In [7]:
aiml_spec = """
<aiml version="0.1">

<output>
    <object name="bank_run" format="length: 2">
        <string
            name="explanation"
            description="A paragraph about what a bank run is."
            format="length: 200 240"
            on-fail-length="reask"
        />
        <url
            name="follow_up_url"
            description="A web URL where I can read more about bank runs."
            required="true"
            format="valid-url"
            on-fail-valid-url="filter"
        />
    </object>
</output>

<prompt>
Explain what a bank run is in a tweet.

@xml_prefix_prompt

{output_schema}

@json_suffix_prompt_v2_wo_none
</prompt>
</aiml>
"""

with tempfile.NamedTemporaryFile(mode="w", suffix=".aiml") as f:
    f.write(aiml_spec)
    f.flush()
    guard_with_reask = gd.Guard.from_aiml(f.name)

raw_llm_output, validated_output = guard_with_reask(openai.Completion.create, engine="text-davinci-003", max_tokens=1024, temperature=0.3)

# Print the validated output from the LLM
print(validated_output)

{
    'bank_run': {
        'explanation': 'A bank run is when a large number of people withdraw their deposits from a bank due to 
concerns about its solvency. This can cause a financial crisis if the bank is unable to meet the demand for 
withdrawals.',
        'follow_up_url': 'https://www.investopedia.com/terms/b/bankrun.asp'
    }
}

In [8]:
print(f'Len of explanation: {len(validated_output["bank_run"]["explanation"])}')

Len of explanation: 207